# Metric Presentation and Visualization
## Necessary packages and functions call

- DDPM-TS: Interpretable Diffusion for Time Series Generation
- Metrics: 
    - discriminative_metrics
    - predictive_metrics
    - visualization

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

## Data Loading

Load original dataset and preprocess the loaded data.

In [2]:
iterations = 5
dataset_name = 'energy'
seq_length = 49
# ori_data = np.load('../toy_exp/samples/energy_ground_truth_24_train.npy')
# ori_data = np.load(f'../energy_results/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
ori_data = np.load(f'../energy_results/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')
fake_data = np.load('../energy_results/ddpm_fake_energy_0_to_1.npy')

## Evaluate the generated data

### 1. Discriminative score

To evaluate the classification accuracy between original and synthetic data using post-hoc RNN network. The output is | classification accuracy - 0.5 |.

- metric_iteration: the number of iterations for metric computation.

In [3]:
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:ori_data.shape[0]])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('energy:')
display_scores(discriminative_score)
print()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 2000/2000 [00:27<00:00, 73.64it/s]


Iter 0:  0.05657894736842106 , 0.5289473684210526 , 0.35789473684210527 



training: 100%|██████████| 2000/2000 [00:26<00:00, 74.40it/s]


Iter 1:  0.030263157894736825 , 0.5289473684210526 , 0.4105263157894737 



...

Iter 2:  0.035526315789473684 , 0.5263157894736842 , 0.4026315789473684 



training: 100%|██████████| 2000/2000 [00:26<00:00, 74.66it/s]


Iter 3:  0.025000000000000022 , 0.5789473684210527 , 0.37105263157894736 



training: 100%|██████████| 2000/2000 [00:26<00:00, 74.67it/s]


Iter 4:  0.022368421052631593 , 0.48947368421052634 , 0.46578947368421053 

energy:
Final Score:  0.033947368421052636 ± 0.016915618793326297



## Evaluate the generated data

### 2. Predictive score

To evaluate the prediction performance on train on synthetic, test on real setting. More specifically, we use Post-hoc RNN architecture to predict one-step ahead and report the performance in terms of MAE. 

The model learns to predict the last dimension with one more step.

In [4]:
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:ori_data.shape[0]])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('energy:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [00:46<00:00, 107.32it/s]


0  epoch:  0.020726414330019476 



training: 100%|██████████| 5000/5000 [00:46<00:00, 106.70it/s]


1  epoch:  0.019230194637323075 



training: 100%|██████████| 5000/5000 [00:47<00:00, 104.54it/s]


2  epoch:  0.020761826203082007 



training: 100%|██████████| 5000/5000 [00:45<00:00, 109.83it/s]


3  epoch:  0.019197622605648796 



training: 100%|██████████| 5000/5000 [00:46<00:00, 106.58it/s]


4  epoch:  0.020495461393571394 

energy:
Final Score:  0.02008230383392895 ± 0.0009925702150199297

